# Compare Analysis

This file is used to compare outputs.

## 1. Setting up the networks to analyse

In [ ]:
import os

if not os.getcwd().endswith("pypsa-eur-climact"):
    %cd ../..
%pwd

In [ ]:
from pathlib import Path
import pypsa
import pandas as pd
import pylab
import re

%matplotlib inline
pylab.rcParams["figure.figsize"] = (25, 6)
pd.set_option("display.width", 1000)

netlist = {
    "VEKA_av": {
        "path": Path("analysis", "VEKA_av_bio_fix_nuc_bev", "results"),
        "fn": "elec_s181_37m_lv3.0__3H-I-T-H-B-A_",
    },
    "VEKA_el": {
        "path": Path("analysis", "VEKA_el_bio_fix_nuc_bev", "results"),
        "fn": "elec_s181_37m_lv3.0__3H-I-T-H-B-A_",
    },
}

year = 2050
netlist = {f"{k} {year}": v for k, v in netlist.items()}

In [ ]:
font = 13
def graph_display(n: dict[int, pd.DataFrame], kind: str = "plot", title: str = "", unit: str = ""):
    df = []
    for k, ni in n.items():
        ax = df.append(ni.sum(axis=1).rename(f"{k}"))
    df = pd.concat(df, axis=1)
    if kind == "plot":
        ax = df.sort_index().fillna(method="ffill").plot()
        ax.set_ylabel(unit, fontsize=font)
        ax.set_title(title, fontsize=font)
    elif kind == "bar":
        ax = df.plot.bar()
        ax.set_ylabel(unit, fontsize=font)
        ax.set_title(title, fontsize=font)
    elif kind == "hist":
        ax = df.plot.hist(alpha=0.8)
        ax.set_xlabel(unit, fontsize=font)
        ax.set_title(title, fontsize=font)
    else:
        raise RuntimeError
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

def get_state_of_charge_t(n, carrier):
    df = n.storage_units_t.state_of_charge.T.reset_index()
    df = df.merge(n.storage_units.reset_index()[["carrier", "StorageUnit"]], on="StorageUnit")
    df = df.groupby(by="carrier").sum()
    del df["StorageUnit"]
    return df.T[[carrier]]

def get_e_t(n, carrier):
    df = n.stores_t.e.T.reset_index()
    df = df.merge(n.stores.reset_index()[["carrier", "Store"]], on="Store")
    df = df.groupby(by="carrier").sum()
    del df["Store"]
    return df.T[[carrier]]

In [ ]:
n = {}
for k, v in netlist.items():
    n[k] = pypsa.Network(Path(v["path"], "postnetworks", v["fn"] + f"{year}.nc"))

## 2. Evaluate renewables potential

In [ ]:
renamer = {"offwind-dc": "offwind", "offwind-ac": "offwind", "solar rooftop": "solar", "coal": "coal/lignite", "lignite": "coal/lignite",
"residential rural biomass boiler": "residential / services biomass boiler",
"residential urban decentral biomass boiler": "residential / services biomass boiler",
"services rural biomass boiler": "residential / services biomass boiler",
"services urban decentral biomass boiler": "residential / services biomass boiler",
"residential rural gas boiler": "residential / services gas boiler",
"residential urban decentral gas boiler": "residential / services gas boiler",
"services rural gas boiler": "residential / services gas boiler",
"services urban decentral gas boiler": "residential / services gas boiler",
"urban central gas boiler": "residential / services gas boiler",
"residential rural water tanks charger": "residential / services water tanks charger",
"residential urban decentral water tanks charger": "residential / services water tanks charger",
"services rural water tanks charger": "residential / services water tanks charger",
"services urban decentral water tanks charger": "residential / services water tanks charger",
"urban central water tanks charger": "residential / services water tanks charger",
"residential rural water tanks discharger": "residential / services water tanks discharger",
"residential urban decentral water tanks discharger": "residential / services water tanks discharger",
"services rural water tanks discharger": "residential / services water tanks discharger",
"services urban decentral water tanks discharger": "residential / services water tanks discharger",
"urban central water tanks discharger": "residential / services water tanks discharger",
"residential rural ground heat pump": "residential / services rural ground heat pump",
"services rural ground heat pump": "residential / services rural ground heat pump",
"residential rural oil boiler": "residential / services oil boiler",
"residential urban decentral oil boiler": "residential / services oil boiler",
"services rural oil boiler": "residential / services oil boiler",
"services urban decentral oil boiler": "residential / services oil boiler",
"urban central oil boiler": "residential / services oil boiler",
"residential rural resistive heater": "residential / services resistive heater",
"residential urban decentral resistive heater": "residential / services resistive heater",
"services rural resistive heater": "residential / services resistive heater",
"services urban decentral resistive heater": "residential / services resistive heater",
"urban central resistive heater": "residential / services resistive heater",
"residential rural solar thermal": "residential / services solar thermal",
"residential urban decentral solar thermal": "residential / services solar thermal",
"services rural solar thermal": "residential / services solar thermal",
"services urban decentral solar thermal": "residential / services solar thermal",
"urban central solar thermal": "residential / services solar thermal",
"residential urban decentral air heat pump": "residential / services air heat pump",
"services urban decentral air heat pump": "residential / services air heat pump",
"urban central air heat pump": "residential / services air heat pump",
}

In [ ]:
rx = re.compile("([A-z]+)[0-9]+\s[0-9]+\s([A-z\-\s]+)-*([0-9]*)")
dimensions = ["region", "carrier", "build_year"]

dfx = []
for run, ni in n.items():
    df_max = pd.DataFrame(ni.generators.p_nom_max)
    df_opt = pd.DataFrame(ni.generators.p_nom_opt)
    df = df_max.join(df_opt).reset_index()
    df[dimensions] = df["Generator"].str.extract(rx)
    df["carrier"] = df["carrier"].str.rstrip("-").replace(renamer)
    df["run"] = run
    df = df[df["carrier"].isin(["onwind", "offwind", "solar"])]
    dfx.append(df.groupby(["run", "carrier", "build_year"]).sum(numeric_only=True)/1e3) #GW

dfx = pd.concat(dfx)
df_potential = pd.concat([
    dfx.loc[dfx["p_nom_opt"].index.get_level_values("build_year") != str(year), "p_nom_opt"].groupby(["run", "carrier"]).sum(), 
    dfx.loc[dfx["p_nom_max"].index.get_level_values("build_year") == str(year), "p_nom_max"].groupby(["run", "carrier"]).sum()
    ], axis=1)
df_potential.fillna(0, inplace=True)
df_potential["potential"] = df_potential["p_nom_opt"] + df_potential["p_nom_max"]
df_potential = df_potential.reset_index().pivot(index="carrier", columns="run", values="potential")
df_potential.plot(kind="bar", title="Renewables potential", ylabel="GW")

## 3. Displaying installed capacities for power supply units

In [ ]:
var = "p_nom_opt"
balance_exclude = ["H2 Electrolysis", "H2 Fuel Cell", "battery charger", "battery discharger", "home battery charger", "home battery discharger", "Haber-Bosch", "Sabatier", "ammonia cracker", "helmeth", "SMR", "SMR CC", "BEV charger", "biogas to gas", "biomass to liquid", "Fischer-Tropsch", "methanolisation", "V2G"]
carriers_links = ["coal", "lignite", "oil", "biomass"] # same carrier name than link
carriers = carriers_links + ["gas", "uranium", "BioSNG", "gas for industry", "gas for industry CC", "process emissions", "process emissions CC", "solid biomass for industry", "solid biomass for industry CC"] # different carrier name than link
transmissions = ["DC", "gas pipeline", "gas pipeline new", "CO2 pipeline", "H2 pipeline", "H2 pipeline retrofitted", "electricity distribution grid", "solid biomass transport"]
balance_carriers_transmission_exclude = balance_exclude + carriers + transmissions

n_links = {}
for run, ni in n.items():
    # Grab data from various sources
    n_run = pd.concat([
        ni.links.groupby(by="carrier").sum(),
        ni.generators.groupby(by="carrier").sum(), 
        ni.storage_units.groupby(by="carrier").sum()
    ])
    n_run = n_run.rename(index=renamer)
    n_run = n_run[~n_run.index.isin(balance_carriers_transmission_exclude)]

    # Grab exceptions for carriers/links   
    n_y_except = ni.links.groupby(by="carrier").sum()
    n_y_except = n_y_except.rename(index=renamer)
    n_y_except = n_y_except[n_y_except.index.isin(carriers_links)]

    n_links[run] = pd.concat([n_run, n_y_except])
    
graph_display({k: ni.groupby(by="carrier").sum()[[var]]/1e3 for k, ni in n_links.items()}, kind="bar", title="Optimised capacity for active power for various planning horizons", unit="GW")

In [ ]:
n_links_display = pd.DataFrame()
for k, v in n_links.items():
    payload = v[[var]].groupby(by="carrier").sum().rename(columns={var: k})/1e3  #GW
    if n_links_display.empty:
        n_links_display = payload
    else:
        n_links_display = n_links_display.join(payload, on="carrier", how="outer")
n_links_display["sum"] = n_links_display.sum(axis=1, numeric_only=True)
n_links_display.sort_values(by="sum", ascending=False).dropna().round(2)[n_links.keys()]

## 4. Displaying installed capacities for power balance units

In [ ]:
n_storage = {}
for run, ni in n.items():
    n_run = pd.concat([ni.links.groupby(by="carrier").sum(), ni.generators.groupby(by="carrier").sum()])
    n_run = n_run.rename(index=renamer)
    n_run = n_run[n_run.index.isin(balance_exclude)]
    n_storage[run] = n_run
graph_display({k: ni.groupby(by="carrier").sum()[[var]]/1e3 for k, ni in n_storage.items()}, kind="bar", title="Optimised capacity for active power for various planning horizons", unit="GW")

In [ ]:
n_storage_display = pd.DataFrame()
for k, v in n_storage.items():
    payload = v[[var]].groupby(by="carrier").sum().rename(columns={var: k})/1e3  #GW
    if n_storage_display.empty:
        n_storage_display = payload
    else:
        n_storage_display = n_storage_display.join(payload, on="carrier", how="outer")
n_storage_display["sum"] = n_storage_display.sum(axis=1, numeric_only=True)
n_storage_display.sort_values(by="sum", ascending=False).dropna().round(2)[n_links.keys()]

## 5. Displaying optimal capacity of transmission network

In [ ]:
capacity = []

length = [] # seems to be wrong estimation

count = []

titles = []

for run, ni in n.items():
    titles.append(run)
    capacity.append(
        (ni.lines.s_nom_opt.sum()
        +
        ni.links[ni.links.carrier=="DC"].p_nom_opt.sum())
        /1e3
    )
    length.append(
        (ni.lines.length.sum()
         +
         ni.links[ni.links.carrier=="DC"].length.sum()
         )/1e3
    )
    count.append(
        len(ni.lines.index)
        +
        len(ni.links[ni.links.carrier=="DC"].p_nom_opt.index)
    )

df = pd.DataFrame([capacity, length, count], index=["capacity [GVA]", "length [1000 km]", "count"], columns=titles)
df.plot(kind="bar", title="AC + DC transmission network")

print(df)



## 6. Displaying loads on various carriers

In [ ]:
carriers_execpt_elec = ["H2", "NH3", "heat", "oil", "gas", "kerosene", "solid biomass", "process emissions", "naphtha", "methanol"]
graph_display({y: pd.DataFrame((ni.loads_t.p.loc[:,~(ni.loads_t.p.columns.str.contains('|'.join(carriers_execpt_elec)))]).sum(axis=1))/1e3 for y, ni in n.items()}, title="Electrical load on the network", unit="GW")
for carrier in carriers_execpt_elec:
    if carrier != "process emissions":
        graph_display({y: pd.DataFrame((ni.loads_t.p.loc[:,(ni.loads_t.p.columns.str.contains(carrier))& ~(ni.loads_t.p.columns.str.contains("emissions"))]).sum(axis=1))/1e3 for y, ni in n.items()}, title=f"{carrier.capitalize()} load on the network", unit="GW")

## 7. Displaying annual profiles for storage units

In [ ]:
for carrier in ["hydro", "PHS"]:
    graph_display({k: get_state_of_charge_t(ni, carrier)/1e6 for k, ni in n.items()}, title=f"State of charge of {carrier}", unit="TWh")
for carrier in ["H2 Store", "battery", "home battery", "co2 stored", "ammonia store"]:
    graph_display({k: get_e_t(ni, carrier)/1e6 for k, ni in n.items()}, title=f"State of charge of {carrier}", unit=r"Mt$_{co2}$"if carrier =="co2 stored" else "TWh")

## 8. Displaying profiles for load and generation

In [ ]:
graph_display({y: pd.DataFrame(ni.links_t.p1[[c for c in ni.links_t.p1.columns if "CGT" in c]].sum(axis=1))/-1e3 for y, ni in n.items()}, title="Electricity provided by gas", unit="GW")
graph_display({y: pd.DataFrame(ni.generators_t.p[[c for c in ni.generators_t.p.columns if "wind" in c]].sum(axis=1))/1e3 for y, ni in n.items()}, title="Electricity provided by wind", unit="GW")
graph_display({y: pd.DataFrame(ni.generators_t.p[[c for c in ni.generators_t.p.columns if "solar" in c]].sum(axis=1))/1e3 for y, ni in n.items()}, title="Electricity provided by solar", unit="GW")
graph_display({y: pd.DataFrame(ni.links_t.p1[[c for c in ni.links_t.p1.columns if "H2 Fuel Cell" in c]].sum(axis=1))/-1e3 for y, ni in n.items()}, title="Electricity provided by fuel cell", unit="GW")
graph_display({y: pd.DataFrame(ni.links_t.p1[[c for c in ni.links_t.p1.columns if ("battery discharger" in c) or ("V2G" in c)]].sum(axis=1))/-1e3 for y, ni in n.items()}, title="Electricity provided by batteries / V2G", unit="GW")

graph_display({y: pd.DataFrame(ni.links_t.p1[[c for c in ni.links_t.p1.columns if "H2 Electrolysis" in c]].sum(axis=1))/-1e3 for y, ni in n.items()}, title="Hydrogen provided by Electrolysis", unit="GW")
graph_display({y: pd.DataFrame(ni.links_t.p1[[c for c in ni.links_t.p1.columns if "Haber-Bosch" in c]].sum(axis=1))/-1e3 for y, ni in n.items()}, title="Ammonia provided by Haber-Bosch", unit="GW")